# Project: Performance of phenotype algorithms for the identification of opioid-exposed infants, Andrew D. Wiese et al. Hospital Pediatrics 2024
# Title: Identify infants with positive opioid toxicology results
# Summary: 
## Identify infants with positive opioid toxicology results



##### Algorihtm steps
```
1.READ infant_tox_lab_list table
2.READ measurement table
3.FILTER measurement table WHERE measurement_source_value IN infant_tox_lab_list short and long names
4.UNION filtered measurement tables
5.FILTER union result WHERE value_source_value is 'present' or 'positive' 
6.JOIN filtered result with mom_baby_step1 on person_id
7.SAVE joined result as infant_tox_lab view
```

##### Data Dictionaries
**infant_tox_lab_list** - Table containing lab test short and long names  
**meas_table** - Measurement table containing lab test results  
**mom_baby_step1** - Table containing mom and baby person IDs

**infant_tox_lab** - View containing baby person IDs and their positive lab test results

##### Usage Notes
```
- The algorithm filters the measurement table twice - first to get measurements matching the infant_tox_lab_list, and again to filter on positive value_source_value
- The filtered measurement tables are unioned before the second filter
- The positive lab results are inner joined with mom_baby_step1 to get the associated baby person IDs
- The final view infant_tox_lab can be used to identify babies with positive lab test results
```

In [0]:
%run "./project_modules"

##### Tox codes-names: phenotyping.mprint_infant_tox_lab_name_sheet_4
##### Search in measurement table to get lab value for baby
##### the lab value (value_source_value) need to be 'present' or 'positive'

In [0]:
sql=f"select * from {infant_tox_lab_list};"
spark.sql(sql).display()

In [0]:
sql=f"""
       select b.FACT_ID_2 as baby_person_id, allrecord.* from 
       (
           select * from 
          (
           SELECT * FROM {meas_table} WHERE lower(trim(measurement_source_value)) in 
           (SELECT lower_lab_short_name FROM {infant_tox_lab_list})
           union
           SELECT * FROM {meas_table} WHERE lower(trim(measurement_source_value)) in 
           (SELECT lower_lab_long_name FROM {infant_tox_lab_list})
           ) as union_result
           where lower(value_source_value) = 'present' or lower(value_source_value) = 'positive'
        ) as allrecord
        inner join 
        global_temp.mom_baby_step1 b
        on allrecord.person_id = b.FACT_ID_2 ;
    """

infant_tox_lab = spark.sql(sql)
infant_tox_lab.name='infant_tox_lab'
register_parquet_global_view(infant_tox_lab)

##### Validation

In [0]:
df_inspection("global_temp.infant_tox_lab","baby")